# Univariate Lift Calculations
This notebook is meant to be a simple example of performing dynamic univariate lift calculations. Given a dataset of observations, we'll designate one column as our outcome and evaluate the rest as univariate features. For each feature in the dataset, we will identify the top 20 most common values and calculate lift for each.

To use this in your own analysis, you'll obviously need a CSV data file with your own observations. Be prepared to make some edits in the below, but the main analysis stage (identified below) should be completely generic and not need any editing. 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Reading Input Data
This open dataset provided by Airbnb (http://data.insideairbnb.com/united-states/nc/asheville/2019-02-17/data/listings.csv.gz) provides some detailed data for just over 2,000 rental listings. We're using only a subset of the columns and will use the rating score as our outcome.

Note that the original file must be unzipped (gzip).

In [3]:
input_filepath = "C:/Users/jacob.carey/Desktop/DataSets/GlobalClimateChangeData/GlobalLandTemperaturesByMajorCity.csv"
dataset = pd.read_csv(input_filepath, header=0, true_values = ['t'], false_values = ['f'],
    usecols=['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country', 'Latitude', 'Longitude'])

In [4]:
dataset.index

RangeIndex(start=0, stop=239177, step=1)

## Data Cleanup
* For sake of ease, let's reduce the review score from a 0-100 range to a boolean. Any score over 95 will be considered a positive rating
* Create buckets out of pricing values, in $50 increments

In [5]:
dataset['id'] = dataset.index

In [15]:
# def is_positive_rating(x):
#     if x >= 95: 
#         return True
#     return False

# dataset['positive_rating'] = dataset['review_scores_rating'].apply(is_positive_rating)
# dataset = dataset.drop(columns=['review_scores_rating'])

In [16]:
# def create_price_group(x):
#     price_int = float(str(x).replace('$','').replace(',',''))
#     price_group = int(price_int / 50) * 50
#     return price_group

# dataset['price_group'] = dataset['price'].apply(create_price_group)
# dataset = dataset.drop(columns=['price'])

In [6]:
dataset.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,id
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W,0
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W,1
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W,2
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W,3
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W,4


## Lift Analysis
Now, we'll iterate over each column to get some counts (totals and positive outcomes) per value in that column. Finally, we'll use these counts to get outcome probabilities per value and calculate lift per value.

In [18]:
outcome_col = 'Consent_bool'
id_col = 'id'
feature_cols = [
    #'ConsentDurationMonths', 
    #'ConsentRequired', 
    'ConsentType', 
    #'Consent_bool', 
    'Country', 
    'LanguageStyle', 
    #'NavigateConsentEnabled', 
    'OnBrand', 
    #'PageClickConsentEnabled', 
    #'ScrollConsentEnabled'
                ]


**NOTE!!!** Everything in this next code block is completely generic. No edits should be required!

In [19]:

# Need some global counts
total_count = len(dataset.index)
total_positives = len(dataset[(dataset[outcome_col] == True)])

# An empty list to hold the count results
count_list = list()

# Iterate over each feature to collect individual counts per value
for feature in feature_cols:
    counts_df = pd.DataFrame(dataset.groupby([feature,])[id_col].count()).nlargest(20,[id_col])  # only top 20
    counts_df.rename({id_col: 'count'}, axis='columns', inplace=True)
    positives_df = pd.DataFrame(dataset[(dataset[outcome_col] == True)].groupby([feature,])[id_col].count())  # all values
    positives_df.rename({id_col: 'positives'}, axis='columns', inplace=True)
    # merge these dataframes
    merge_df = counts_df.merge(positives_df, left_index=True, right_index=True)
    # iterate over rows, building a dict per row, and append each to list of counts
    for this_row in merge_df.iterrows():
        this_dict = {
            'feature': feature,
            'value': str(this_row[0]),
            'total_count': total_count,
            'total_positives': total_positives,
            'count': this_row[1]['count'],
            'positives': this_row[1]['positives']
        }
        count_list.append(this_dict)

# Create a new dataframe from the aggregated list
lift_df = pd.DataFrame(count_list)

# Now let's add some calculations for probabilities and lift per row
lift_df['total_prob'] = lift_df['total_positives'] / lift_df['total_count']
lift_df['prob'] = lift_df['positives'] / lift_df['count']
lift_df['lift - more likely'] = lift_df['prob'] / lift_df['total_prob']
lift_df['lift - less likely'] = 1 / lift_df['lift - more likely']
lift_df['prct_total'] = lift_df['count'] / lift_df['total_count']

In [20]:
lift_df

,count,feature,positives,total_count,total_positives,value,total_prob,prob,lift - more likely,lift - less likely,prct_total
0,69035451,ConsentType,33565964,85832663,47347945,2,0.551631,0.486213,0.881411,1.134545,0.804303
1,15222590,ConsentType,12588495,85832663,47347945,3,0.551631,0.826961,1.499121,0.667058,0.177352
2,1574622,ConsentType,1193486,85832663,47347945,0,0.551631,0.757951,1.374018,0.727792,0.018345
3,44036479,Country,23528550,85832663,47347945,2,0.551631,0.534297,0.968577,1.032443,0.513050
4,21356769,Country,16820142,85832663,47347945,1,0.551631,0.787579,1.427728,0.700413,0.248819
5,20439415,Country,6999253,85832663,47347945,3,0.551631,0.342439,0.620776,1.610888,0.238131
6,48792506,LanguageStyle,22135286,85832663,47347945,1,0.551631,0.453662,0.822401,1.215952,0.568461
7,29391304,LanguageStyle,20073835,85832663,47347945,99,0.551631,0.682986,1.238121,0.807676,0.342426
8,7648853,LanguageStyle,5138824,85832663,47347945,0,0.551631,0.671842,1.217920,0.821072,0.089114
9,49497168,OnBrand,23731853,85832663,47347945,2,0.551631,0.479459,0.869166,1.150528,0.576671


## Output CSV
Push this final lift dataframe to a csv file for use elsewhere...

In [21]:
date_time = datetime.today().strftime('%m-%d-%H%M')

In [22]:
output_filepath = 'C:/Users/jacob.carey/Desktop/SiteNotice Consent Rate Analysis/univariate_results/univariate_results_fullset_{0}.csv'.format(date_time)

lift_df.to_csv(output_filepath, index=False)

&copy; Mackinac Data Group, 2019